**Модель с аугментацией и повышенным штрафом за 2 класса**

In [ ]:
import os
import shutil
from google.colab import drive

# Монтирование Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import shutil
import os

# Это уже аугментированный датасет. Путь к зазипованному датасету и директория, куда его нужно распаковать
zip_path = '/content/drive/MyDrive/aug_dataset_1.2.zip'
extract_path = '/content/aug_dataset_1.2'

# Распаковка ZIP-файла
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Путь для копирования в /content/
local_zip_path = '/content/aug_dataset_1.2.zip'

# Копирование файла
shutil.copy(zip_path, local_zip_path)

# Проверка успешного копирования
if os.path.exists(local_zip_path):
    print(f"Файл {local_zip_path} успешно скопирован.")
else:
    print(f"Ошибка при копировании файла.")



Файл /content/aug_dataset_1.2.zip успешно скопирован.


In [ ]:
import os

def count_files_in_directory(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files

# Путь к папке
dataset_path = '/content/aug_dataset_1.2'

# Подсчет всех файлов во всех вложенных папках
total_files = count_files_in_directory(dataset_path)

# Вывод количества файлов
print(f"Всего файлов в папке {dataset_path} и её вложенных папках: {total_files}")


Всего файлов в папке /content/aug_dataset_1.2 и её вложенных папках: 48348


In [ ]:
# Импорт необходимых библиотек для работы с YOLO и обучения модели
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Импорт библиотек для работы с файловой системой и визуализации
import os
import matplotlib.pyplot as plt

print("Библиотеки успешно импортированы.")


Библиотеки успешно импортированы.


In [ ]:

!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 17125, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 17125 (delta 9), reused 26 (delta 6), pack-reused 17085
Receiving objects: 100% (17125/17125), 9.02 MiB | 23.61 MiB/s, done.
Resolving deltas: 100% (11967/11967), done.


In [ ]:

!pip install ultralytics==8.0.20
from IPython import display
display.clear_output()

In [ ]:
# Нам надо поменять ямл - пути и ввести штрафы. сначала смотрим, акакой он сейчас
import yaml

with open("/content/data6.yaml", 'r') as stream:
    try:
        data = yaml.safe_load(stream)
        print("Пути в YAML-файле после обновления:")
        print(data)
    except yaml.YAMLError as exc:
        print(exc)


Пути в YAML-файле после обновления:
{'names': ['Agricultural object', 'Fields', 'Forest', 'Grassland', 'Industrial facility', 'Power lines', 'Public facility', 'Residential facility', 'Road', 'Water'], 'nc': 10, 'test': '/content/dataset_1.0/dataset_1.0/test/images', 'test_labels': '/content/dataset_1.0/dataset_1.0/test/labels', 'train': '/content/dataset_1.0/dataset_1.0/train/images', 'train_labels': '/content/dataset_1.0/dataset_1.0/train/labels', 'val': '/content/dataset_1.0/dataset_1.0/valid/images', 'val_labels': '/content/dataset_1.0/dataset_1.0/valid/labels'}


In [ ]:
pip install PyYAML


In [ ]:
import yaml

# Ямл меняем под новый датасет
yaml_path = '/content/data6.yaml'

# Чтение YAML-файла
with open(yaml_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Назначаем повышенные штрафы для недостаточных классов 4,5,6,7 и меняем пути
yaml_data['names'] = ['Agricultural object', 'Fields', 'Forest', 'Grassland', 'Industrial facility', 'Power lines', 'Public facility', 'Residential facility', 'Road', 'Water']
yaml_data['nc'] = 10  # количество классов
yaml_data['cls_pw'] = [1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0]  # веса классов
yaml_data['train'] = '/content/aug_dataset_1.2/train/images'
yaml_data['train_labels'] = '/content/aug_dataset_1.2/train/labels'
yaml_data['val'] = '/content/aug_dataset_1.2/valid/images'
yaml_data['val_labels'] = '/content/aug_dataset_1.2/valid/labels'

# Удаление ключей для тестовых данных, если их нет (в новом датасете 1.2. нет папок тест, значит, эти пути убираем)
if 'test' in yaml_data:
    del yaml_data['test']
if 'test_labels' in yaml_data:
    del yaml_data['test_labels']

# Сохранение изменений в том же YAML-файле
with open(yaml_path, 'w') as file:
    yaml.safe_dump(yaml_data, file)



In [ ]:
from imgaug import augmenters as iaa
import cv2
import os
import shutil

# Если датасет уже аугментирован, эту ячейку не запускать!
# Была попытка аугментировать все классы, но колаб не тянет такой датасет. в итоге это аугментация только файлов с объектами класса индастриал.
# Гипотеза, что так можно улучшить очень низкий уровень дифференциации с классом резиденшиал. Выбран только поворот под углом.
# Путь к папке с тренировочными изображениями и масками
image_path = '/content/dataset_1.2/train/images'
mask_path = '/content/dataset_1.2/train/labels'

# ID класса "индастриал" в data.yaml
industrial_class_id = 3  # Замените на актуальный ID

# Аугментатор для поворота
seq = iaa.Sequential([
    iaa.Affine(rotate=(-25, 25))
])

# Функция для аугментации изображений и копирования масок
def augment_images(image_path, mask_path, num_augmentations=1):
    for filename in os.listdir(image_path):
        img = cv2.imread(os.path.join(image_path, filename))
        mask_filename = filename.split('.')[0] + '.txt'

        # Проверка, содержит ли маска класс "индастриал"
        with open(os.path.join(mask_path, mask_filename), 'r') as f:
            lines = f.readlines()
            if any(str(industrial_class_id) == line.split()[0] for line in lines):
                for i in range(num_augmentations):
                    images_aug = seq.augment_image(img)

                    # Сохранение аугментированных изображений
                    cv2.imwrite(os.path.join(image_path, f"aug_{i}_{filename}"), images_aug)

                    # Копирование соответствующих масок
                    new_mask_filename = f"aug_{i}_{mask_filename}"
                    shutil.copy(os.path.join(mask_path, mask_filename), os.path.join(mask_path, new_mask_filename))

# Применение аугментации
augment_images(image_path, mask_path)



In [ ]:
# Ячейку не запускать при наличии готового АУГ-датасета. Проверяем, сколько всего получилось файлов и вес датасета.
# Инициализация счетчика аугментированных файлов
augmented_count = 0

# Подсчет аугментированных файлов
for filename in os.listdir(image_path):
    if "aug_" in filename:
        augmented_count += 1

# Вывод количества аугментированных файлов
print(f"Total number of augmented files: {augmented_count}")

# Вывод веса датасета (в байтах)
dataset_size = sum(os.path.getsize(os.path.join(dirpath, filename)) for dirpath, dirnames, filenames in os.walk('/content/dataset_1.2') for filename in filenames)
print(f"Total dataset size: {dataset_size} bytes")



Total number of augmented files: 7064
Total dataset size: 3043152110 bytes


In [ ]:
import os

# Ячейку не запускать при наличсии АУГ-датасета. Путь к папке с тренировочными изображениями
image_path = '/content/dataset_1.2/train/images'

# Получение списка всех файлов в директории
all_files = os.listdir(image_path)

# Фильтрация списка, чтобы оставить только аугментированные файлы
augmented_files = [f for f in all_files if f.startswith("aug_")]

# Вывод первых 10 аугментированных файлов
print(augmented_files[:10])


['aug_0_DS1_6-7_3614.jpg', 'aug_0_DS1_6-7_5350.jpg', 'aug_0_DS1_13_258.jpg', 'aug_0_DS1_6-7_4779.jpg', 'aug_0_DS1_11-1_142.jpg', 'aug_0_DS1_12_274.jpg', 'aug_0_DS1_11-1_139.jpg', 'aug_0_DS1_6-7_5330.jpg', 'aug_0_DS1_6-7_2333.jpg', 'aug_0_DS1_6-7_4934.jpg']


In [ ]:
import zipfile
# Распаковка готовго ауг-датасета
zip_path = '/content/aug_dataset_1.2.zip'
extract_path = '/content/aug_dataset_1.2'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
import yaml
#забыла подправить пути под ауг-датасет
yaml_file_path = '/content/data6.yaml'

# Чтение текущего YAML-файла
with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Обновление путей к изображениям и меткам
yaml_data['train'] = '/content/aug_dataset_1.2/train/images'
yaml_data['train_labels'] = '/content/aug_dataset_1.2/train/labels'
yaml_data['val'] = '/content/aug_dataset_1.2/valid/images'
yaml_data['val_labels'] = '/content/aug_dataset_1.2/valid/labels'

# Сохранение обновленного YAML-файла
with open(yaml_file_path, 'w') as file:
    yaml.safe_dump(yaml_data, file)

In [ ]:
import fiftyone as fo

# При наличии платформы 51, не исполнялись в данном колабе. Создание нового датасета
dataset = fo.Dataset(name="my_dataset")

# Загрузка изображений из директории
dataset.add_images("/path/to/image/directory")

# Загрузка данных
sample = fo.Sample(filepath="/path/to/image.jpg")
sample["label"] = fo.Classification(label="cat")
dataset.add_sample(sample)

sample.tags.append("train")
sample["custom_field"] = 51

dataset.add_sample(sample)

view = dataset.match_tags("train").sort_by("custom_field").limit(10)

for sample in view:
    print(sample)

In [ ]:
import fiftyone.brain as fob
# При наличии платформы 51, не исполнялись в данном колабе.
fob.compute_uniqueness(dataset)
rank_view = dataset.sort_by("uniqueness")

In [ ]:
# Запуск сессии для визуализации
# При наличии платформы 51, не исполнялись в данном колабе. Создание нового датасета
session = fo.launch_app(dataset)

In [ ]:
#импортируем предобученную модель
from ultralytics import YOLO

model5 = YOLO('/content/yolov8s-seg.pt')
# Display model information (optional)
model5.info()


100%|██████████| 22.8M/22.8M [00:00<00:00, 174MB/s]

YOLOv8s-seg summary: 261 layers, 11821056 parameters, 0 gradients, 42.9 GFLOPs


In [ ]:
#тренируем модель, в установках показывает в колабе прогресс по каждой эпохе
results = model5.train(data='/content/data6.yaml', epochs=20, imgsz=640, save=True, project='/content/runs', verbose=True)



Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8s-seg.yaml, data=/content/data6.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=/content/runs, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False

In [ ]:
#Валидация.
from ultralytics import YOLO


# Validate the model
model5.val()  # оцените производительность модели на наборе проверки
metrics = model5.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps   # a list contains map50-95(B) of each category
metrics.seg.map    # map50-95(M)
metrics.seg.map50  # map50(M)
metrics.seg.map75  # map75(M)
metrics.seg.maps   # a list contains map50-95(M) of each category

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8s-seg summary (fused): 195 layers, 11783470 parameters, 0 gradients, 42.5 GFLOPs
val: Scanning /content/dataset_1.0/dataset_1.0/valid/labels.cache... 1435 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1435/1435 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/dataset_1.0/dataset_1.0/valid/images/DS1_6-7_2270.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 90/90 [00:32<00:00,  2.77it/s]
                   all       1435       4732      0.763        0.6       0.67      0.541      0.744      0.589      0.641      0.471
   Agricultural object       1435        173      0.858       0.85      0.923      0.827      0.868      0.861      0.922      0.823
                Fields       1435        661      0.924      0.877      0.924      0.856      0.923      0.

AttributeError: ignored

In [ ]:
model5.predict(source="/content/plant_test.jpg", save=True)  # предсказать по изображению


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8s-seg summary (fused): 195 layers, 11783470 parameters, 0 gradients, 42.5 GFLOPs
Results saved to /content/runs/predict


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([11, 6])
 dtype: torch.float32
  + tensor([[3.39000e+02, 1.38000e+02, 3.96000e+02, 2.05000e+02, 6.87915e-01, 7.00000e+00],
         [3.77000e+02, 1.71000e+02, 5.82000e+02, 3.68000e+02, 6.20954e-01, 7.00000e+00],
         [4.13000e+02, 9.90000e+01, 4.74000e+02, 1.63000e+02, 5.94115e-01, 7.00000e+00],
         [5.34000e+02, 1.27000e+02, 6.27000e+02, 2.12000e+02, 5.37429e-01, 7.00000e+00],
         [5.55000e+02, 3.26000e+02, 6.22000e+02, 3.93000e+02, 4.14397e-01, 7.00000e+00],
         [1.84000e+02, 2.75000e+02, 2.30000e+02, 3.16000e+02, 4.09055e-01, 7.00000e+00],
         [3.00000e+00, 0.00000e+00, 7.14000e+02, 4.95000e+02, 3.79945e-01, 4.00000e+00],
         [4.84000e+02, 3.90000e+01, 5.17000e+02, 7.10000e+01, 3.76819e-01, 7.00000e+00],
         [2.66000e+02, 1.06000e+02, 3.44000e+02, 1.70000e+02, 2.84687e-01, 7.00000e+00],
         [1.00000e+00, 4.03000e+02, 7.99000e

In [ ]:
model5.predict(source="/content/hotel.jpg", save=True)  # предсказать по изображению


Results saved to /content/runs/predict


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
 type: <class 'torch.Tensor'>
 shape: torch.Size([12, 6])
 dtype: torch.float32
  + tensor([[2.00000e+02, 1.00000e+00, 8.87000e+02, 2.29000e+02, 8.60971e-01, 2.00000e+00],
         [4.00000e+00, 5.70000e+01, 4.03000e+02, 5.41000e+02, 6.46077e-01, 2.00000e+00],
         [1.00000e+00, 2.00000e+01, 1.24000e+02, 2.06000e+02, 6.37445e-01, 7.00000e+00],
         [3.24000e+02, 5.48000e+02, 5.22000e+02, 6.66000e+02, 5.65429e-01, 2.00000e+00],
         [0.00000e+00, 4.67000e+02, 3.10000e+02, 6.64000e+02, 5.26230e-01, 7.00000e+00],
         [1.87000e+02, 3.61000e+02, 2.28000e+02, 3.97000e+02, 5.17698e-01, 7.00000e+00],
         [1.19000e+02, 3.31000e+02, 1.59000e+02, 3.72000e+02, 4.05388e-01, 7.00000e+00],
         [9.00000e+01, 3.18000e+02, 1.60000e+02, 3.67000e+02, 3.84132e-01, 7.00000e+00],
         [7.23000e+02, 5.11000e+02, 8.89000e+02, 6.66000e+02, 3.77528e-01, 2.00000e+00],
         [9.10000e+01, 3.18000e+02, 1.29000e

In [ ]:

# Define source as YouTube video URL
source = 'https://www.youtube.com/watch?v=PlPlB9C5CYE'

# Run inference on the source
model5.predict(source='https://www.youtube.com/watch?v=PlPlB9C5CYE', save=True)  # предсказать по изображению
results=model5


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

requirements: YOLOv8 requirements "pafy" "youtube_dl==2020.12.2" not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.3 MB/s eta 0:00:00

requirements: 2 packages updated per ('pafy', 'youtube_dl==2020.12.2')
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

1/1: https://www.youtube.com/watch?v=PlPlB9C5CYE...  Success (540 frames 640x360 at 23.98 FPS)

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
Results saved to /content/runs/predict


In [ ]:
import cv2
import matplotlib.pyplot as plt


# Define source as YouTube video URL
#Отчет в ранс показал, что данное видео в отличие от предыдущего не прочиталось!
source = '/content/jj-autumn-fall-aerial.mp4, save=True'

# Run inference on the source
model5.predict(source='/content/jj-autumn-fall-aerial.mp4', save=True)  # предсказать по изображению
results=model5


Results saved to /content/runs/predict


In [ ]:
import zipfile

# Путь для упаковки, перенесем всю папку ранс на гугл драйв
folder_path = '/content/runs/'

# имя архива
zip_file = '/content/runs5.zip'

# Создайте архив
with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, folder_path)
            zipf.write(file_path, arcname=arcname)

print(f"Архив {zip_file} успешно создан.")


Архив /content/runs5.zip успешно создан.


In [ ]:
import shutil

# Путь к созданному вами архиву
zip_file_path = '/content/runs5.zip'

# Путь к месту на Google Drive, куда хотите переместить архив
drive_destination_path = '/content/drive/My Drive/'

# Скопируйте архив на Google Drive
shutil.copy(zip_file_path, drive_destination_path)

print(f"Архив {zip_file_path} успешно скопирован на Google Drive в папку {drive_destination_path}.")




Архив /content/runs5.zip успешно скопирован на Google Drive в папку /content/drive/My Drive/.


In [ ]:
# Добавление предсказаний модели
# при наличии платформы 51, в данном коде не запускалась
sample["prediction"] = fo.Classification(label="dog", confidence=0.9)
sample.save()

# Использование встроенных функций для анализа
results = dataset.evaluate_classifications("predictions", gt_field="label", eval_key="eval")
